In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
from polyglot.downloader import downloader
downloader.download("embeddings2.en")
downloader.download("embeddings2.hi")
downloader.download("embeddings2.ta")
downloader.download("transliteration2.hi")
downloader.download("transliteration2.ta")

[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package embeddings2.en is already up-to-date!
[polyglot_data] Downloading package embeddings2.hi to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package embeddings2.hi is already up-to-date!
[polyglot_data] Downloading package embeddings2.ta to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package embeddings2.ta is already up-to-date!
[polyglot_data] Downloading package transliteration2.hi to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package transliteration2.hi is already up-to-date!
[polyglot_data] Downloading package transliteration2.ta to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package transliteration2.ta is already up-to-date!


True

In [3]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "ta", split="train+validation")
common_voice_test = load_dataset("common_voice", "ta", split="test")

Reusing dataset common_voice (/home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/7cd6a2cd99f885b3ec1205a6aee65d9b8c7b36a2c0f482fa4a1dde3d29860f21)
Reusing dataset common_voice (/home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/7cd6a2cd99f885b3ec1205a6aee65d9b8c7b36a2c0f482fa4a1dde3d29860f21)


In [4]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [5]:
import re
import polyglot
from polyglot.text import Text
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'


def get_transliterations(batch):
    batch["sentence"] = Text(batch["sentence"], hint_language_code="ta")
    batch["sentence"] = (" ".join(list(batch["sentence"].transliterate("en"))))
    return batch

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).upper() + " "
    return batch

In [6]:
common_voice_train = common_voice_train.map(get_transliterations)
common_voice_test = common_voice_test.map(get_transliterations)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/7cd6a2cd99f885b3ec1205a6aee65d9b8c7b36a2c0f482fa4a1dde3d29860f21/cache-40c1b13abdf9a963.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/7cd6a2cd99f885b3ec1205a6aee65d9b8c7b36a2c0f482fa4a1dde3d29860f21/cache-b0d3a16d14456a07.arrow


In [7]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/7cd6a2cd99f885b3ec1205a6aee65d9b8c7b36a2c0f482fa4a1dde3d29860f21/cache-9079bae2db1769b3.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/7cd6a2cd99f885b3ec1205a6aee65d9b8c7b36a2c0f482fa4a1dde3d29860f21/cache-3aab2a7283d95814.arrow


In [8]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("config/vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token="|")

In [9]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [10]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [11]:
common_voice_train[0]["audio"]

{'path': 'cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19093432.mp3',
 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00031871,
        -0.00020725, -0.00080252], dtype=float32),
 'sampling_rate': 48000}

In [12]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [13]:
common_voice_train[0]["audio"]

{'path': 'cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19093432.mp3',
 'array': array([ 0.        ,  0.        ,  0.        , ...,  0.0005847 ,
         0.00018592, -0.00039355], dtype=float32),
 'sampling_rate': 16000}

In [14]:
import random
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text:    WANGAI VAND VDAIAIUM BITTAIUM  
Input array shape: (66432,)
Sampling rate: 16000


In [15]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [16]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/3788 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/7cd6a2cd99f885b3ec1205a6aee65d9b8c7b36a2c0f482fa4a1dde3d29860f21/cache-4cefba3a1d1ed5e0.arrow


In [17]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [18]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [19]:
wer_metric = load_metric("wer")

In [20]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [21]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.weight', 'project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [22]:
model.freeze_feature_extractor()
model.gradient_checkpointing_enable()

/home/ubuntu/environments/r-u/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1674: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-large-rbg-tamil",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [ ]:
trainer.train()

/home/ubuntu/environments/r-u/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3788
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 5900


***** Running Evaluation *****
  Num examples = 1781
  Batch size = 8
Saving model checkpoint to wav2vec2-large-rbg-tamil/checkpoint-100
Configuration saved in wav2vec2-large-rbg-tamil/checkpoint-100/config.json
Model weights saved in wav2vec2-large-rbg-tamil/checkpoint-100/pytorch_model.bin
Feature extractor saved in wav2vec2-large-rbg-tamil/checkpoint-100/preprocessor_config.json
